In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-classification-2024-spring:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F78272%2F8577682%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240614%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240614T073446Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dae29bf1b8438cdfccbbd16b53f5105f025a9f1a9104a7d119e56633501b9ce2fceb0233adbda03940a7be45d74b7b6f2b05d702484bd09481960b3a7f0c8bf01e3df0a4276641748c9d5abd6b546f09523acedc17f93c4c4c136f83346a610a7a0a7041368b0d0251cf0cc27f76fda6182017f5fa6970fda1c6916c8a90901e9b204934a33e967bd7c0257c42135dc500fbfde200cf5001c5ce7b1bfb5080e92e7ba33d02c3f464185e896b21fb9cdc091315b41c94576e741bc8aaacb85cca105fc123a368fbc60020b2c11e245889edc6aebd37d4687b0deb45826e3e3ea21bed51bdf8098749b825a830b40d3b280a44b35cfccd1f03e37012f8c79849ded'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1250295019 bytes downloaded
Downloaded and uncompressed: image-classification-2024-spring
Data source import complete.


In [2]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split

from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import os
import random
from tqdm import tqdm
from PIL import Image

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(7)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


In [6]:
class JpegCompression(transforms.Lambda):
    def __init__(self, quality_lower=60, quality_upper=100, p=0.5):
        super().__init__(self.apply_jpeg_compression)
        self.quality_lower = quality_lower
        self.quality_upper = quality_upper
        self.probability = p

    def apply_jpeg_compression(self, img):
        if random.random() < self.probability:
            quality = random.randint(self.quality_lower, self.quality_upper)
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=quality)
            buffer.seek(0)
            img = Image.open(buffer)
        return img

In [7]:
def random_color_jitter():
    return transforms.ColorJitter(
        brightness=random.uniform(0.1, 0.3),
        contrast=random.uniform(0.1, 0.3),
        saturation=random.uniform(0.1, 0.3)
    )

In [8]:
class ConditionalResize:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        if max(img.size) < self.target_size:
            return transforms.Resize(self.target_size, interpolation=transforms.InterpolationMode.BILINEAR)(img)
        return img

In [9]:
import torchvision.transforms as transforms
import io
from torchvision.transforms.functional import to_pil_image, to_tensor

# 필요한 크기
target_size = 380

transform = transforms.Compose([
    ConditionalResize(target_size),  # target_size보다 작은 경우에만 리사이즈
    transforms.CenterCrop(target_size),  # 모든 이미지를 target_size로 중앙 자르기
    JpegCompression(quality_lower=60, quality_upper=90, p=0.2),
    transforms.RandomApply([  # 색상 조정을 랜덤하게 적용
        transforms.Lambda(lambda img: random_color_jitter()(img))
    ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
dataset = datasets.ImageFolder(root='/kaggle/input/image-classification-2024-spring/dataset/train', transform=transform)

In [11]:
dataset_size = len(dataset)
train_size = int(dataset_size * 0.95)
val_size = dataset_size - train_size

trainset, valset = random_split(dataset, [train_size, val_size])

In [12]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(valset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
import torchvision.models as models
# EfficientNet V2-S 모델을 불러옵니다. 미리 학습된 가중치를 사용합니다.
model = models.efficientnet_v2_s(pretrained=True)

num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

# model = nn.DataParallel(model, device_ids=[0, 1])
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 151MB/s]


In [14]:
from torch.optim import SGD
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=40, gamma=0.5)

In [15]:
if not os.path.exists('checkpoint'):
    os.makedirs('checkpoint')

best_acc = 0.

In [16]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    preds = []
    labels = []

    for inputs, label in tqdm(train_loader):
        inputs = inputs.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        preds += predicted.detach().cpu().numpy().tolist()
        labels += label.detach().cpu().numpy().tolist()

    train_accuracy = accuracy_score(labels, preds)
    print(f'train_accuracy: {train_accuracy}')

    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for inputs, label in tqdm(val_loader):
            inputs = inputs.to(device)
            label = label.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            preds += predicted.detach().cpu().numpy().tolist()
            labels += label.detach().cpu().numpy().tolist()

    val_accuracy = accuracy_score(labels, preds)
    print(f'val_accuracy: {val_accuracy}')

    if epoch == 14:
        best_acc = val_accuracy
        torch.save(model.state_dict(), 'checkpoint/model5.pth')


 81%|████████  | 482/594 [07:45<01:48,  1.04it/s]


KeyboardInterrupt: 